In [ ]:
##1. plotting all the restos on a map

In [ ]:
!pip install pandas folium

In [ ]:
import pandas as pd
import os

# Use the lines=True parameter to read JSON Lines format
df = pd.read_json('/Users/comanetan/Downloads/yelp_academic_dataset_restaurants.json', lines=True)

# Display the first 5 rows of the DataFrame
print(df.head())


In [ ]:
# check for null values in lat and long
print("Checking for null values before cleaning:")
print(df[['latitude', 'longitude']].isnull().sum())

# remove rows with null values. create a new DataFrame that drops rows where either 'latitude' or 'longitude' is missing.
df_cleaned = df.dropna(subset=['latitude', 'longitude'])

# cleaning verifcation, how many rows removed
print(f"\nOriginal DataFrame shape: {df.shape}")
print(f"Cleaned DataFrame shape:  {df_cleaned.shape}")


In [ ]:
import folium

# find center of the map
map_center = [df['latitude'].mean(), df['longitude'].mean()]

# Create a map object
m = folium.Map(location=map_center, zoom_start=11)

# Display the map 
m

In [ ]:
# Create a new map object
m_points = folium.Map(location=map_center, zoom_start=11)

# Add a marker for each restaurant
for idx, row in df.head(60000).iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=3,
        color='#3186cc',
        fill=True,
        fill_color='#3186cc',
        # Add a popup with the restaurant's name
        popup=row['name']
    ).add_to(m_points)

# Display the map
m_points

In [ ]:
## Create heatmap
from folium.plugins import HeatMap

# Create a list of [latitude, longitude] pairs
heat_data = df[['latitude', 'longitude']].values.tolist()

# Create a new map
m_heatmap = folium.Map(location=map_center, zoom_start=11)

# Add the heatmap layer
HeatMap(heat_data).add_to(m_heatmap)

# Display the map
m_heatmap

In [ ]:
### i find this heatmapping a bit hard to read, but i think its a good start?

In [ ]:
## trying market clustering, groups nearby points into a single, numbered circle. zooming in, clusters will break apart into smaller clusters or individual points
from folium.plugins import MarkerCluster

# Calculate the center of the map
map_center = [df_cleaned['latitude'].mean(), df_cleaned['longitude'].mean()]

# Create a new map object
m_cluster = folium.Map(location=map_center, zoom_start=11)

# Create a MarkerCluster object
marker_cluster = MarkerCluster().add_to(m_cluster)

# Add points to the cluster
for idx, row in df_cleaned.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['name']  # Optional: show name on click
    ).add_to(marker_cluster)

# Display the map
m_cluster

In [ ]:
###exploring narrowing it down further to quality restos

# Filter for restaurants with 4.5 stars or more and a sufficient reviews
df_top_rated = df_cleaned[(df_cleaned['stars'] >= 4.5) & (df_cleaned['review_count'] > 50)]
print(f"Found {len(df_top_rated)} top-rated restaurants.")

# Create a new map for the top-rated spots
m_top_rated = folium.Map(location=map_center, zoom_start=11)

for idx, row in df_top_rated.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"{row['name']} ({row['stars']} stars)",
        icon=folium.Icon(color='green', icon='star') # Add a green star icon
    ).add_to(m_top_rated)

# Display the map
m_top_rated

In [ ]:
##### Further analysis: eda on which city has the most restos?

In [ ]:
!pip install matplotlib seaborn

In [ ]:
import pandas as pd

# split the 'categories' string into a list of categories
df_cleaned['category_list'] = df_cleaned['categories'].str.split(', ')

# 2. create a new row for each category in the list.
df_exploded = df_cleaned.explode('category_list')

print("Data after exploding categories:")
print(df_exploded[['name', 'category_list']].head())

In [ ]:
# Get a list of unique cities in the dataset
print("Cities available in the dataset:")
print(df_cleaned['city'].unique())

In [ ]:
## EDA on top cities with most no. of businesses on yelp 
import matplotlib.pyplot as plt
import seaborn as sns

# Get count of businesses for each city and select the top 20
top_20_cities = df_cleaned['city'].value_counts().head(20)

print("Top 20 Cities by Number of Businesses:")
print(top_20_cities)

# Create a bar plot to visualize the results
plt.figure(figsize=(12, 10)) # Adjust figure size for better readability
sns.barplot(x=top_20_cities.values, y=top_20_cities.index, palette='rocket')

plt.title('Top 20 Cities by Number of Businesses', fontsize=16)
plt.xlabel('Number of Businesses', fontsize=12)
plt.ylabel('City', fontsize=12)
plt.xticks(rotation=45) # Rotate x-axis labels slightly if they overlap
plt.tight_layout() # Adjust plot to ensure everything fits without overlapping

plt.show()

In [ ]:
# (bonus) more biz analysis: which cuisine is popular in which town?

In [ ]:
## with the above, i narrowed the analysis to top 10 cities 

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


top_cities_list = [
    'Philadelphia',
    'Tampa',
    'Indianapolis', 
    'Nashville',
    'Tucson',
    'New Orleans',
    'Edmonton',
    'Saint Louis',
    'Reno',
    'Boise'
]


# Define generic categories to exclude for more meaningful results
generic_categories = ['Restaurants', 'Food', 'Shopping', 'Beauty & Spas', 'Nightlife', 'Bars', 'Event Planning & Services']

print("--- Starting Cuisine Analysis for Top 10 Cities ---")

for city in top_cities_list:
    # Filter for the current city
    city_df = df_exploded[df_exploded['city'] == city]

    if city_df.empty:
        print(f"\nWarning: No data found for '{city}'. It might be misspelled or not in the dataset.")
        continue

    # Filter out generic categories
    city_df_filtered = city_df[~city_df['category_list'].isin(generic_categories)]

    # Get the top 10 most common cuisines/categories
    top_cuisines = city_df_filtered['category_list'].value_counts().head(10)

    # get plot
    plt.figure(figsize=(12, 8))
    sns.barplot(x=top_cuisines.values, y=top_cuisines.index, palette='plasma')

    plt.title(f'Top 10 Most Popular Cuisines in {city}', fontsize=18)
    plt.xlabel('Number of Businesses', fontsize=12)
    plt.ylabel('Cuisine / Category', fontsize=12)
    plt.tight_layout()


    plt.show()


In [ ]:
## i then tried to make some meaningful clusters, bc the map and heatmap did not seem too helpful

In [ ]:
!pip install scikit-learn

In [ ]:
# Create a new map
map_clusters = folium.Map(location=[df_cleaned['latitude'].mean(), df_cleaned['longitude'].mean()], zoom_start=11)

# --- Create a color palette for the clusters ---
# We create a list of colors to cycle through for each cluster
from colorsys import hsv_to_rgb
cluster_colors = [f"#{int(r*255):02x}{int(g*255):02x}{int(b*255):02x}" for r,g,b in [hsv_to_rgb(x/n_clusters, 1., 1.) for x in range(n_clusters)]]

# --- Loop through and plot each point with its cluster color ---
for idx, row in df_cleaned.iterrows():
    if row['cluster'] != -1: # If the point is part of a cluster
        color = cluster_colors[row['cluster']]
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=4,
            popup=f"Cluster {row['cluster']}",
            color=color,
            fill=True,
            fill_color=color
        ).add_to(map_clusters)

# Display the final map with clusters
map_clusters

In [ ]:
## 2. DOES LOCATION AFFECT PRICE??


In [ ]:
import pandas as pd
import json
import os

# --- Step 1: Check if the file exists and provide the correct path ---
file_path = 'yelp_food_business_flat.json'  # Update this with the correct path if needed

# Check if the file exists before trying to open it
if not os.path.exists(file_path):
    print(f"Error: The file '{file_path}' does not exist.")
    print(f"Current working directory: {os.getcwd()}")
    print("Please check the file path or upload the file to this directory.")
else:
    # --- Step 2: Load the raw JSON data from the file ---
    with open(file_path, 'r') as f:
        data = json.load(f)

    # --- Step 3: Use json_normalize to flatten the data correctly ---
    df_normalized = pd.json_normalize(data)

    print("Successfully loaded and flattened the data!")
    print("Here are the available columns now:")
    print(df_normalized.columns.tolist())

    # --- Step 4: Clean the data (remove null lat/lon) ---
    df_cleaned = df_normalized.dropna(subset=['latitude', 'longitude'])